In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [4]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [5]:
# z-score normalization
scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [10]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[-1.28200357  0.06173483 -1.28187449 -0.41449729  0.46579774 -1.17798747
  0.06471169 -0.85753636 -0.64435287 -1.27747438]
[-1.13903374 -0.75002097 -1.13420654 -0.43244806 -0.38227578 -1.17669163
  0.04549002 -0.83947808 -0.63455673 -1.16125476]


In [11]:
X = train_x_list
y = train_y_list

In [12]:
# SVM 
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

clf = svm.SVC().fit(X, y)
print clf
predict_y_list = clf.predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-26 16:21:00
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.49601373496253531, 0.28445302519810611, 0.29807435040057706, None)
weighted:  (0.61544487999173569, 0.66513261475112029, 0.59908376494444471, None)
end: 2016-12-26 16:22:13


In [13]:
# SVM with class_weight='balanced'
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

clf = svm.SVC(class_weight='balanced').fit(X, y)
print clf
predict_y_list = clf.predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-26 16:22:13
SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.33471047772992807, 0.41088171030458698, 0.35022002324791102, None)
weighted:  (0.60335579154366337, 0.5066004602155747, 0.5389480771387325, None)
end: 2016-12-26 16:23:45


In [14]:
# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

# SVM with SMOTE up-sample
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

clf = svm.SVC().fit(train_x_list_smote, train_y_list_smote)
print clf
predict_y_list = clf.predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

No handlers could be found for logger "imblearn.base"


number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590
start: 2016-12-26 16:23:55
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35106291678333362, 0.37477248438018795, 0.35799155522823456, None)
weighted:  (0.58661081201509746, 0.5772072181179605, 0.58046592334128633, None)
end: 2016-12-26 16:28:21


In [15]:
print predict_y_list[:100]

[ 2.  2.  2.  2.  2.  0.  2.  2.  2.  2.  3.  2.  2.  3.  2.  1.  2.  2.
  3.  2.  2.  2.  4.  1.  2.  2.  2.  2.  2.  4.  2.  2.  0.  2.  2.  1.
  1.  2.  2.  2.  2.  2.  0.  2.  2.  1.  1.  1.  1.  1.  2.  2.  2.  4.
  1.  2.  2.  2.  0.  2.  2.  2.  4.  2.  0.  1.  1.  2.  3.  2.  2.  2.
  1.  1.  1.  2.  1.  2.  1.  1.  1.  1.  1.  2.  2.  1.  1.  1.  1.  1.
  1.  4.  2.  2.  2.  2.  2.  1.  1.  1.]


In [16]:
print test_y_list[:100]

[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 2 2 1 1 2 1 1 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2
 1 1 1 2 1 1 1 1 1 2 2 1 1 1 2 1 1 2 2 2 2 2 1 1 1 2]
